### Anime Recommendation System

In [44]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

In [9]:
anime = pd.read_csv("./data.csv")
anime.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [10]:
anime.isnull().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    0
dtype: int64

In [21]:
anime['info'] = anime.apply(lambda row: f"Title: {row['Name']}. Genre: {row['Genres']}. Overview: {row['sypnopsis']}", axis=1) # combine everything

In [22]:
anime.head()

,MAL_ID,Name,Score,Genres,sypnopsis,info
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...","Title: Cowboy Bebop. Genre: Action, Adventure,..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ...",Title: Cowboy Bebop: Tengoku no Tobira. Genre:...
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0...","Title: Trigun. Genre: Action, Sci-Fi, Adventur..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...,"Title: Witch Hunter Robin. Genre: Action, Myst..."
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...,"Title: Bouken Ou Beet. Genre: Adventure, Fanta..."


In [23]:
anime[['info']].to_csv("anime_info.csv", index=False)

**Data Loader and Vector Store**

In [30]:
loader = CSVLoader(file_path="./anime_info.csv" , encoding="utf-8")

In [35]:
data = loader.load()

In [33]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [36]:
texts = text_splitter.split_documents(data)

In [40]:
embeddings = HuggingFaceEmbeddings()

C:\Users\samjh\AppData\Local\Temp\ipykernel_752\3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
D:\Programming\LangChain\Anime Recommendation System\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\Programming\LangChain\Anime Recommendation System\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\samjh\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a

In [48]:
load_dotenv()

API_KEY = os.getenv("API_KEY")

MODEL = "llama-3.3-70b-versatile"

In [49]:
llm = ChatGroq(api_key=API_KEY, model=MODEL, temperature=0)

In [50]:
doc_search = Chroma.from_documents(texts, embeddings)

**Querying VectorDB Store**

In [51]:
query = "I am looking for an animated sci-fi movie. What could you suggest to me?"

In [57]:
docs = doc_search.similarity_search(query, k=1)

In [58]:
docs

[Document(metadata={'source': './anime_info.csv', 'row': 161}, page_content='info: Title: Xenosaga The Animation. Genre: Action, Mecha, Sci-Fi, Shounen, Space. Overview: The year is T.C. 4767. Four thousand years have passed since humanity abandoned its birthplace, the planet Earth. Beset by the hostile alien Gnosis, mankind is now scrambling to find ways to defeat this threat to their existence. The development of KOS-MOS (a specialized android with amazing capabilities) by Vector engineer Shion Uzuki was one response to the threat. But when their ship is destroyed by the Gnosis, Shion and her companions find themselves thrust into the middle of a battle with no clear sides... (Source: Anime-Planet)')]

**Using QA Retrieval**

In [59]:
qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=doc_search.as_retriever(), return_source_documents=True)

In [61]:
query = "Can you recommend me a sci-fi anime with a unique storyline?"
result = qa.invoke({"query": query})

In [62]:
result['result']

'Based on the information I have, I\'d like to recommend "Xenosaga The Animation" or "Divergence Eve 2: Misaki Chronicles". Both of these anime series have unique storylines that involve sci-fi elements, space, and mecha.\n\n"Xenosaga The Animation" is set in a future where humanity has abandoned Earth and is fighting against hostile aliens known as the Gnosis. The story follows Shion Uzuki, an engineer who develops a powerful android named KOS-MOS, and her companions as they become embroiled in a battle with no clear sides.\n\n"Divergence Eve 2: Misaki Chronicles", on the other hand, involves a complex storyline with time travel, alternate timelines, and multiple versions of the main character, Misaki. The series explores themes of identity, reality, and the consequences of playing with time.\n\nBoth series have intricate plots and explore complex ideas, making them great recommendations for someone looking for a unique sci-fi anime storyline.'

In [63]:
result['source_documents'][0]

Document(metadata={'row': 161, 'source': './anime_info.csv'}, page_content='info: Title: Xenosaga The Animation. Genre: Action, Mecha, Sci-Fi, Shounen, Space. Overview: The year is T.C. 4767. Four thousand years have passed since humanity abandoned its birthplace, the planet Earth. Beset by the hostile alien Gnosis, mankind is now scrambling to find ways to defeat this threat to their existence. The development of KOS-MOS (a specialized android with amazing capabilities) by Vector engineer Shion Uzuki was one response to the threat. But when their ship is destroyed by the Gnosis, Shion and her companions find themselves thrust into the middle of a battle with no clear sides... (Source: Anime-Planet)')

**Prompt Engineering**

In [64]:
from langchain.prompts import PromptTemplate

In [67]:
template = """
You are an anime recommenation system that helps users find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason.
If you dont know, just say so, dont try to make up an answer.

{context}

Question: {question}
Your response: 
"""

In [68]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [69]:
chain_type_kwargs = {"prompt": prompt}

In [70]:
qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=doc_search.as_retriever(), return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

In [71]:
query = "Can you recommend me an anime about Pirates?"
result = qa.invoke({"query": query})

In [73]:
print(result['result'])

Based on your interest in pirates, I'd like to recommend the following three anime:

1. **One Piece**: This anime is a classic pirate-themed adventure that follows Monkey D. Luffy and his crew as they travel across the Grand Line in search of the ultimate treasure, One Piece. With a mix of action, comedy, and fantasy, One Piece is a great fit for anyone looking for a pirate-themed anime.
2. **Vandread**: While not exclusively a pirate anime, Vandread features a group of female pirates who play a central role in the story. The anime combines elements of science fiction, action, and comedy, and explores the themes of cooperation and understanding between different groups.
3. **Ou Dorobou Jing**: Although not traditional pirates, Jing and his partner Kir can be considered a type of "space pirate" or thief. The anime features a mix of adventure, comedy, and fantasy, and follows Jing's exploits as he travels from town to town, stealing valuable items and using his cunning and skills to outw

**New Template**

In [74]:
template_prefix = """
You are an anime recommenation system that helps users find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason.
If you dont know, just say so, dont try to make up an answer.

{context}
"""


user_info = """
This is what we know about the user, and you can use this information to better recommendations
Age: {age}
Gender: {gender}
"""

template_suffix = """
Question: {question}
Your response: 
"""

user_info = user_info.format(age=18, gender="male")

Combined_prompt = template_prefix + "\n" + user_info + "\n" + template_suffix

print(Combined_prompt)


You are an anime recommenation system that helps users find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason.
If you dont know, just say so, dont try to make up an answer.

{context}


This is what we know about the user, and you can use this information to better recommendations
Age: 18
Gender: male


Question: {question}
Your response: 



In [76]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": prompt}

qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=doc_search.as_retriever(), return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

query = "Can you recommend me an anime about Ninjas?"
result = qa.invoke({"query": query})

print(result['result'])

A great request! Based on your interest in ninjas, I'd like to recommend the following three anime:

1. **Ninin ga Shinobuden**: This anime is a comedy-focused series that parodies the ninja genre. It follows the story of Kaede, a normal school girl, and Shinobu, a ninja-in-training, as they form an unexpected friendship. The series is light-hearted and humorous, making it a great choice for those looking for a fun ninja-themed anime.
2. **Basilisk: Kouga Ninpou Chou**: This anime is a more serious and action-packed series set in feudal Japan. It follows the story of two ninja clans, the Iga and Kouga, as they engage in a bitter war. The series explores themes of love, duty, and loyalty, making it a great choice for those looking for a more dramatic ninja anime.
3. **Rekka no Honoo**: This anime follows the story of Rekka, a self-styled modern-day ninja who discovers he's the last surviving member of a legendary ninja clan. The series combines action, adventure, and martial arts, makin